In [ ]:
import igraph as ig
import pandas as pd
import json
import random

In [ ]:
directory = "deezer_clean_data/"
edges_file = "RO_edges.csv"      # CSV com as amizades
genres_file = "RO_genres.json" # JSON com os gêneros
sample_size = 2000

In [ ]:

# ============================================================
# 1️⃣  Amostragem por BFS (Breadth-First Search)
# ============================================================

def bfs_sample(graph: ig.Graph, sample_size: int, start_vertex=None):
    """
    Retorna um subgrafo conexo criado via expansão BFS a partir de um vértice inicial.
    """
    if start_vertex is None:
        start_vertex = random.randrange(graph.vcount())

    bfs_order = graph.bfs(start_vertex)[0]  # lista de nós visitados na ordem BFS
    sample_nodes = bfs_order[:sample_size]
    subgraph = graph.subgraph(sample_nodes)

    return subgraph


# ============================================================
# 2️⃣  Amostragem por Random Walk
# ============================================================

def random_walk_sample(graph: ig.Graph, sample_size: int, start_vertex=None):
    """
    Realiza uma caminhada aleatória pela rede, garantindo conectividade.
    Retorna um subgrafo com 'sample_size' nós visitados e preserva os IDs originais.
    """
    if start_vertex is None:
        start_vertex = random.randrange(graph.vcount())

    visited = {start_vertex}
    current = start_vertex

    while len(visited) < sample_size:
        neighbors = graph.neighbors(current)
        if not neighbors:  # nó isolado
            current = random.choice(list(visited))
            continue
        current = random.choice(neighbors)
        visited.add(current)

    visited = list(visited)

    # Cria subgrafo
    subgraph = graph.subgraph(visited)

    # Armazena o ID original como atributo dos vértices
    subgraph.vs["original_id"] = visited

    return subgraph

# ============================================================
# 3️⃣  Amostragem simples + componente gigante
# ============================================================

def random_component_sample(graph: ig.Graph, sample_size: int):
    """
    Faz uma amostragem aleatória de nós e retorna apenas
    a maior componente conexa do subgrafo.
    """
    if sample_size > graph.vcount():
        raise ValueError("sample_size maior que o número total de nós.")

    sample_nodes = random.sample(range(graph.vcount()), sample_size)
    subgraph = graph.subgraph(sample_nodes)

    # Pega a componente conexa gigante
    components = subgraph.components()
    largest = components.giant()

    return largest

In [ ]:
# --- 1. Ler o CSV de arestas ---
# O arquivo contém pares de IDs (u, v) representando amizades mútuas
edges_df = pd.read_csv(directory + edges_file, header=None, skiprows=1)
edges = list(zip(edges_df[0], edges_df[1]))

# --- 2. Criar grafo não-direcionado ---
G = ig.Graph(edges=edges, directed=False)

print(f"Grafo criado com {G.vcount()} nós e {G.ecount()} arestas")

# --- 3. Ler o JSON com preferências musicais ---
with open(directory + genres_file, "r") as f:
    genres_data = json.load(f)

# --- 4. Adicionar atributo 'genres' a cada nó ---
# Os nós foram indexados de 0 em diante, então o índice do JSON = ID do nó
genres_attr = [None] * G.vcount()  # inicializa lista de atributos

for node_id, liked_genres in genres_data.items():
    node_id = int(node_id)
    if node_id < G.vcount():  # evita erro se houver IDs fora do range
        genres_attr[node_id] = liked_genres

# adiciona o atributo ao grafo
G.vs["genres"] = genres_attr

# --- 5. Exemplo de uso ---
# Exibir informações básicas do grafo
print("\n--- Informações do Grafo ---")
print(G.summary())

# Visualizar os gêneros de um nó qualquer
example_node = 0
print(f"\nUsuário {example_node} gosta de:", G.vs[example_node]["genres"])

In [ ]:
# --- Visualizar parte da rede usando metodos Sample ---

bfs_test = bfs_sample(G, sample_size)
rw_sub = random_walk_sample(G, sample_size)
rc_sub = random_component_sample(G, sample_size)


# ig.plot(bfs_test, vertex_size=5, vertex_color="lightblue")
ig.plot(rw_sub, vertex_size=5, vertex_color="lightblue")
# ig.plot(rc_sub, vertex_size=5, vertex_color="lightblue")

# ------ Resultado: Random Walk Sample é o melhor método

In [ ]:

sample_genres = {
    v["original_id"]: genres_file[str(v["original_id"])]
    for v in rw_sub.vs if str(v["original_id"]) in genres_file
}